In [49]:
import mysql.connector

from IPython.display import display, Markdown, HTML

db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="secret",
    port="3306",
    database="serlo",
    charset="latin1"
)

def query(sql):
    c = db.cursor()
    c.execute(sql)
    
    return c.fetchall()

def querySingleton(sql):
    return [ x[0] for x in query(sql) ]

In [50]:
def query_subjects():
    return query("""
        select term_taxonomy.id, instance.subdomain, term.name
        from term_taxonomy
        join term on term.id = term_taxonomy.term_id
        join term_taxonomy p on p.id = term_taxonomy.parent_id
        join instance on instance.id = term.instance_id
        where p.parent_id is NULL
            or term_taxonomy.parent_id in (106081);
    """)

query_subjects()

[(5, 'de', 'Mathe'),
 (8, 'de', 'Blog'),
 (15465, 'de', 'Mathematik Forum'),
 (17744, 'de', 'Nachhaltigkeit'),
 (17746, 'de', 'Permakultur Forum'),
 (18230, 'de', 'Chemie'),
 (18234, 'de', 'Chemie Forum'),
 (20605, 'de', 'Nutzerprofile, Hilfe, Community und Allgemeines'),
 (20769, 'de', 'Communityseiten - faecheruebergreifend'),
 (23362, 'de', 'Biologie'),
 (23382, 'de', 'Biologie Forum'),
 (23778, 'de', 'Mathematik Forum - test'),
 (24798, 'de', 'dev:blog'),
 (25078, 'de', 'Mathematik Blog'),
 (25712, 'de', 'Deutsch als Fremdsprache'),
 (25979, 'de', 'Englisch'),
 (26523, 'de', 'Betriebswirtschaftslehre mit Rechnungswesen'),
 (26876, 'de', 'Englisch Forum'),
 (26882, 'de', 'Deutsch als Fremdsprache Forum'),
 (33894, 'de', 'Latein'),
 (41107, 'de', 'Physik'),
 (41115, 'de', 'Physik Forum'),
 (47899, 'de', 'Informatik'),
 (59918, 'de', 'Musik'),
 (79159, 'de', 'Politik'),
 (87993, 'de', 'Community'),
 (93176, 'de', 'Testbereich'),
 (181883, 'de', 'Lerntipps'),
 (23593, 'en', 'math'),
 (

In [52]:
bytes('à®\x87à®²à®\x95à¯\x8dà®\x95à®£à®®à¯\x8d', "latin1").decode("utf8")

'இலக்கணம்'

In [15]:
children = {}

for (child_id, parent_id) in query("""select id, parent_id from term_taxonomy;"""):
    if parent_id not in children:
        children[parent_id] = []
    children[parent_id].append(child_id)

children[5]

[1288,
 1289,
 1290,
 1334,
 1339,
 1385,
 1390,
 1397,
 1421,
 16030,
 16063,
 22763,
 22765,
 22951,
 43972,
 44323,
 146728]

In [12]:
entities = {}

for (entity_id, type_name, term_id) in query("""select entity.id, type.name, term_taxonomy_entity.term_taxonomy_id  from term_taxonomy_entity join uuid on uuid.id = term_taxonomy_entity.entity_id join entity on entity.id = term_taxonomy_entity.entity_id join type on type.id
= entity.type_id where uuid.trashed = 0;"""):
    if term_id not in entities:
        entities[term_id] = []
    entities[term_id].append({"id": entity_id, "type": type_name})

entities[24370]

[{'id': 1541, 'type': 'article'},
 {'id': 1543, 'type': 'article'},
 {'id': 1873, 'type': 'article'},
 {'id': 1925, 'type': 'article'},
 {'id': 1949, 'type': 'article'},
 {'id': 35663, 'type': 'article'},
 {'id': 35776, 'type': 'article'},
 {'id': 35866, 'type': 'article'},
 {'id': 35990, 'type': 'text-exercise'},
 {'id': 35991, 'type': 'article'},
 {'id': 36070, 'type': 'article'},
 {'id': 82288, 'type': 'video'},
 {'id': 82294, 'type': 'video'},
 {'id': 82296, 'type': 'video'},
 {'id': 83579, 'type': 'video'}]

In [16]:
def all_entities(term_id):
    if term_id in entities:
        yield from entities[term_id]
        
    if term_id in children:
        for child_id in children[term_id]:
            yield from all_entities(child_id)

list(all_entities(3))[:5]

[{'id': 78176, 'type': 'video'},
 {'id': 1685, 'type': 'article'},
 {'id': 36226, 'type': 'text-exercise'},
 {'id': 44840, 'type': 'video'},
 {'id': 55761, 'type': 'article'}]

In [29]:
import pandas as pd

def df_count_types(subject_id):
    df = pd.DataFrame(all_entities(subject_id))
    if len(df) == 0:
        return df
    df = df.groupby("id").first()
    df.reset_index(inplace=True)
    df = df.groupby("type").count()
    df.rename(columns={"id": "count"}, inplace=True)
    return df

df_count_types(3)

,count
type,
applet,118
article,2374
course,432
event,18
math-puzzle,67
text-exercise,3548
text-exercise-group,1757
video,418


In [54]:
from IPython.display import display, Markdown

def fix_encoding(title):
    return bytes(title, "latin1").decode("utf8")

for (subject_id, instance, name_unfixed) in query_subjects():
    name = fix_encoding(name_unfixed)
    display(Markdown(f"# {instance}: {name} (/{subject_id})"))
    display(df_count_types(subject_id))

# de: Mathe (/5)

,count
type,
applet,113
article,801
course,143
event,1
math-puzzle,63
text-exercise,2950
text-exercise-group,1530
video,335


# de: Blog (/8)

""


# de: Mathematik Forum (/15465)

...

""


# fr: nouvelles matières (/141607)

""


# fr: Conseils pour apprendre (/148619)

,count
type,
article,5
course,3


In [57]:
def save(subject_id, name):
    df = pd.DataFrame(all_entities(subject_id))
    df = df.groupby("id").first()
    df.to_csv(f"/tmp/{name}.csv")

save(41107, "physik")
save(47899, "informatik")